In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import ee
import geemap 
import seaborn as sns
from datetime import datetime

from validate_dowscaled_et import prepare_data_for_plotting

In [11]:
ee.Initialize(project="thurgau-irrigation")

In [12]:
def set_negative_to_zero(image):
    return image.where(image.lt(0), 0)

In [13]:
et_collection_2018 = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_2018")

et_collection_2018.size().getInfo()

1

In [16]:
ee.Date(et_collection_2018.first().date()).format('YYYY-MM-dd').getInfo()

EEException: Image.date: Parameter 'image' is required.

In [15]:
et_collection_2018.first().bandNames().getInfo()

EEException: Image.bandNames: Parameter 'image' is required.

In [6]:
validation_et_collection = ee.FeatureCollection("projects/thurgau-irrigation/assets/Thurgau/ETFluxNet/ET_timeseries_rietholzbach_2009_2015")

In [7]:
validation_et_collection.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [8.998612999999999, 47.3800787405]},
 'id': '0000000000000000000e',
 'properties': {'date_[YYYYmmdd]': '2009-06-15',
  'evapotranspiration_[mm/d]': 0,
  'system:time_start': 1245024000000}}

In [12]:
point = validation_et_collection.first().geometry()

image = et_collection_2018.toList(33).get(25)
Map = geemap.Map()
Map.centerObject(point, 12)
Map.addLayer(point, {"color": "red"}, "point")
options = {
    "bands": ["downscaled"],
    "min": 0,
    "max": 3000,
    "palette": ["blue", "white", "orange", "red"],
}
Map.addLayer(image, options, "ET")
Map

Map(center=[47.3800787405, 8.998612999999999], controls=(WidgetControl(options=['position', 'transparent_bg'],…

In [13]:
image.getInfo()

EEException: List.get: List is empty (index is 25).

In [9]:
df = geemap.ee_to_df(validation_et_collection)
df

,date_[YYYYmmdd],evapotranspiration_[mm/d]
0,2009-06-15,0.0000
1,2009-06-16,0.0000
2,2009-06-20,0.0000
3,2009-06-23,0.0000
4,2009-06-24,0.0000
...,...,...
2401,2011-06-27,6.7195
2402,2015-07-11,6.7520
2403,2012-07-17,6.8603
2404,2012-04-28,6.8789


In [10]:
import ee
import pandas as pd
import matplotlib.pyplot as plt


def extract_pixel_values(
    image_collection: ee.ImageCollection, point: ee.Geometry.Point
) -> ee.FeatureCollection:
    """
    Extract the pixel value of the 'downscaled' band for each image in the collection
    at the specified point.

    Args:
        image_collection (ee.ImageCollection): The input image collection.
        point (ee.Geometry.Point): The point at which to extract values.

    Returns:
        ee.FeatureCollection: A feature collection where each feature represents an image
                              and contains the pixel value of the 'downscaled' band at the point.
    """

    def extract_value(image: ee.Image) -> ee.Feature:
        # Select the 'downscaled' band
        downscaled = image.select("downscaled")

        # Get the scale of the 'downscaled' band
        scale = downscaled.projection().nominalScale()

        # Extract the pixel value at the point
        pixel_value = downscaled.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=scale,
            bestEffort=True,
        ).get("downscaled")

        # Retrieve the image ID (using 'system:index' as an identifier)
        image_id = image.get("system:index")

        # Retrieve the image acquisition time
        time_start = image.get("system:time_start")

        return ee.Feature(
            None,
            {
                "pixel_value": pixel_value,
                "date" : ee.Date(time_start).format("YYYY-MM-dd"),
                "system:time_start": time_start,
            },
        )

    # Map the extraction function over the image collection
    return ee.FeatureCollection(image_collection.map(extract_value))
 

point = validation_et_collection.first().geometry()
raster_collection = extract_pixel_values(et_collection_2018, point)

In [11]:
ee.Date(raster_collection.first().get("system:time_start")).format("YYYY-MM-dd").getInfo()

EEException: Collection.first: Error in map(ID=WaPOR_ET_downscaled_2018-01_D1):
Date: Parameter 'value' is required.

In [12]:
df = geemap.ee_to_df(ee.FeatureCollection(raster_collection))
df

,date,pixel_value
0,2018-01-01,0.000000
1,2018-01-11,0.000000
2,2018-01-21,0.000000
3,2018-02-01,0.000000
4,2018-02-11,0.000000
5,2018-02-21,0.001834
6,2018-03-01,0.000051
7,2018-03-11,0.000079
8,2018-03-21,0.006114
9,2018-04-01,1.001313
